# VLM_SafetyGuidance Quick Start

물류창고 안전 관제 시스템 - 각 모듈 코드 참고

## 1. 전처리 (AI Hub -> YOLO)

In [ ]:
# src/preprocessing/aihub_to_yolo.py 핵심 코드

import json
import shutil
from pathlib import Path

# 카테고리 매핑
CATEGORY_NAMES = {
    "01": "도크설비", "02": "보관", "03": "부가가치서비스",
    "04": "설비및장비", "05": "운반", "06": "입고",
    "07": "지게차", "08": "출고", "09": "파렛트렉",
    "10": "피킹분배", "11": "화재"
}

# 클래스 매핑 (35개)
CLASS_MAPPING = {
    # 안전 객체 (SO)
    "SO-01": 0, "SO-02": 1, "SO-03": 2, "SO-06": 3, "SO-07": 4,
    "SO-08": 5, "SO-12": 6, "SO-13": 7, "SO-14": 8, "SO-15": 9,
    "SO-16": 10, "SO-17": 11, "SO-18": 12, "SO-19": 13, "SO-21": 14,
    "SO-22": 15, "SO-23": 16,
    # 작업 객체 (WO)
    "WO-01": 17, "WO-02": 18, "WO-03": 19, "WO-04": 20,
    "WO-05": 21, "WO-06": 22, "WO-07": 23,
    # 위험 행동 (UA)
    "UA-01": 24, "UA-02": 25, "UA-03": 26, "UA-04": 27,
    "UA-05": 28, "UA-06": 29, "UA-16": 30,
    # 위험 상태 (UC)
    "UC-09": 31, "UC-10": 32, "UC-15": 33, "UC-16": 34
}

def convert_bbox_to_yolo(bbox, img_width, img_height):
    """바운딩박스를 YOLO 형식으로 변환 (normalized x_center, y_center, width, height)"""
    x, y, w, h = bbox
    x_center = (x + w / 2) / img_width
    y_center = (y + h / 2) / img_height
    norm_w = w / img_width
    norm_h = h / img_height
    return x_center, y_center, norm_w, norm_h

In [ ]:
# 전처리 실행 (터미널 명령어)
!cd ../src && python -m preprocessing.aihub_to_yolo --folders 01 --sample 100

## 2. 학습 (YOLO 모델)

In [ ]:
# src/training/train_yolo.py 핵심 코드

from ultralytics import YOLO
from pathlib import Path

class YOLOTrainer:
    def __init__(self, model_name="yolov8n.pt", device="auto"):
        self.model = YOLO(model_name)
        self.device = device
    
    def train(self, category, epochs=100, batch=16, imgsz=640):
        """YOLO 모델 학습"""
        # data.yaml 경로 찾기
        data_yaml = self._find_data_yaml(category)
        
        # 학습 실행
        results = self.model.train(
            data=data_yaml,
            epochs=epochs,
            batch=batch,
            imgsz=imgsz,
            device=self.device,
            project="../models",
            name=f"safety_{category}"
        )
        return results
    
    def _find_data_yaml(self, category):
        """카테고리별 data.yaml 경로 찾기"""
        CATEGORIES = {
            "01": "도크설비", "02": "보관", "03": "부가가치서비스",
            "04": "설비및장비", "05": "운반", "06": "입고",
            "07": "지게차", "08": "출고", "09": "파렛트렉",
            "10": "피킹분배", "11": "화재"
        }
        category_name = CATEGORIES.get(category)
        return f"../data/{category}_{category_name}/logistics_yolo/data.yaml"

In [ ]:
# 학습 실행 (터미널 명령어)
!cd ../src && python -m training.train_yolo --category 01 --epochs 10

## 3. Monitoring (객체 탐지)

In [ ]:
# src/monitoring/__init__.py 핵심 코드

from ultralytics import YOLO
import torch
from pathlib import Path

PROJECT_ROOT = Path("..")
MODELS_DIR = PROJECT_ROOT / "models"

def _find_best_model():
    """models/ 폴더에서 가장 최근 학습된 best.pt 찾기"""
    if not MODELS_DIR.exists():
        return None
    best_models = list(MODELS_DIR.rglob("weights/best.pt"))
    if not best_models:
        return None
    best_models.sort(key=lambda x: x.stat().st_mtime, reverse=True)
    return str(best_models[0])

# 모델 로드
custom_model_path = _find_best_model()
model = YOLO(custom_model_path) if custom_model_path else YOLO('yolov8n.pt')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

# 위험 클래스
ANOMALY_CLASSES = [
    "no_helmet", "no_safety_shoes", "no_safety_vest", "danger_zone_entry",
    "phone_while_driving", "speeding", "other_unsafe_action",
    "pathway_obstacle", "improper_stacking", "poor_lighting", "other_unsafe_condition"
]

## 4. Reasoning (위험 분석)

In [ ]:
# src/reasoning/__init__.py 핵심 코드

import os
import base64
import json
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

def analyze_risk_with_vlm(detection_result):
    """GPT-4o VLM으로 위험 분석"""
    image_path = detection_result.get("image_path")
    with open(image_path, "rb") as f:
        base64_image = base64.b64encode(f.read()).decode('utf-8')
    
    prompt = """
    당신은 공장의 AI 안전 관리자입니다.
    이미지를 분석하여 위험을 평가해주세요.
    - 위험 수준: "LOW", "MED", "HIGH"
    - 이유: 한국어 한 문장
    JSON: {"risk_level": "...", "reason": "..."}
    """
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_image}"}}
            ]
        }],
        response_format={"type": "json_object"}
    )
    return json.loads(response.choices[0].message.content)

## 5. Action (안전 가이드라인 생성)

In [ ]:
# src/action/__init__.py 핵심 코드

def generate_safety_guideline(analysis_result):
    """위험 등급에 따라 다국어 안전 지침 생성"""
    risk_level = analysis_result.get("risk_level")
    reason = analysis_result.get("reason", "")
    
    if risk_level == "LOW":
        return {"status": "logged"}
    
    elif risk_level == "MED":
        print(f"확인 필요: {reason}")
        return {"status": "confirmation_requested"}
    
    elif risk_level == "HIGH":
        # LLM으로 다국어 지침 생성
        prompt = f"""
        상황: {reason}
        다국어 안전 지침 (한국어, 영어, 베트남어)
        JSON: {{"guideline_ko": "...", "guideline_en": "...", "guideline_vi": "..."}}
        """
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            response_format={"type": "json_object"}
        )
        guidelines = json.loads(response.choices[0].message.content)
        print(f"한국어: {guidelines.get('guideline_ko')}")
        print(f"English: {guidelines.get('guideline_en')}")
        print(f"Tiếng Việt: {guidelines.get('guideline_vi')}")
        return {"status": "generated", "guidelines": guidelines}

## 6. 전체 파이프라인 실행

In [ ]:
# 파이프라인 실행
import sys
sys.path.append('../src')

from monitoring import detect_objects
from reasoning import analyze_risk_with_vlm
from action import generate_safety_guideline

# 테스트 이미지
image_path = "../data/01_도크설비/logistics_yolo/val/images/image_000001.jpg"

# 1. 객체 탐지
detection_result = detect_objects(image_path)
print("탐지:", detection_result["status"])

# 2. 위험 분석
if detection_result["status"] == "anomaly_detected":
    analysis_result = analyze_risk_with_vlm(detection_result)
    print("분석:", analysis_result)
    
    # 3. 가이드라인 생성
    if analysis_result:
        generate_safety_guideline(analysis_result)

In [ ]:
# 또는 run.py로 실행
!cd ../src && python run.py --image ../data/01_도크설비/logistics_yolo/val/images/image_000001.jpg